<a href="https://colab.research.google.com/github/LaJeremi/Doit-study-Practice-gichang-Lee-/blob/main/23.%20(%EC%A7%84%ED%96%89%EC%A4%91)%20%20Fine-tuning%20GPT-2%20(Cls%2C%20Chatbot%2C%20NLI)/%2022_03_kogpt2_chatbot_gpu(%EC%8B%A4%ED%8C%A8)_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 22-03 KoGPT-2 텍스트 생성을 이용한 한국어 챗봇

출처: https://github.com/ukairia777/tensorflow-nlp-tutorial/blob/main/22.%20Fine-tuning%20GPT-2%20(Cls%2C%20Chatbot%2C%20NLI)/22-3.%20kogpt2_chatbot_gpu.py

In [ ]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

In [ ]:

tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='', eos_token='', pad_token='')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.5.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'lm_head.weight', 'transformer.h.2.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.6.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expe

In [ ]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)
print('-' * 10)
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))

51200
51200
51200
----------
</s>
<usr>
<pad>
<sys>


In [ ]:
import pandas as pd
import tqdm
import urllib.request

In [ ]:

urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')

In [ ]:
len(train_data)


11823

In [ ]:
batch_size = 32


In [ ]:
def get_chat_data():
  for question, answer in zip(train_data.Q.to_list(), train_data.A.to_list()):
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    sent = tokenizer.encode('' + question + '' + answer) 
    yield bos_token + sent + eos_token

In [ ]:
dataset = tf.data.Dataset.from_generator(get_chat_data, output_types=tf.int32)


In [ ]:
dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)


In [ ]:

for batch in dataset:
    print(batch)
    break

tf.Tensor(
[[51200  9349  7888   739  7318   376 25000  6824  9108  9028  7098 25856
  51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
  51200 51200 51200 51200]
 [51200  9020  8263  7497 10192 11615  8210  8006 11567  8711  9535  7483
  12521 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
  51200 51200 51200 51200]
 [51200  9085  7597   395  8149 10624  7397 24224 13358  7182  8030 19138
  16899  9677  8234   389 51200 51200 51200 51200 51200 51200 51200 51200
  51200 51200 51200 51200]
 [51200  9085  7597   395  8149  9465 10624  7397 24224 13358  7182  8030
  19138 16899  9677  8234   389 51200 51200 51200 51200 51200 51200 51200
  51200 51200 51200 51200]
 [51200  9943   422   418  9327  8702  7098  7141 16912 18328  8671  7415
   8263  8234   389 51200 51200 51200 51200 51200 51200 51200 51200 51200
  51200 51200 51200 51200]
 [51200  9815   410 21249 10174  6824  8210  8006 16146 11056 11594 10137
  10556  9266  8711 25856 51200 51200 51

In [ ]:
tokenizer.decode(batch[0])


'<|endoftext|> 12시 땡!하루가 또 가네요.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>'

In [ ]:
print(batch[0])


tf.Tensor(
[51200  9349  7888   739  7318   376 25000  6824  9108  9028  7098 25856
 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
 51200 51200 51200 51200], shape=(28,), dtype=int32)


In [ ]:
print(tokenizer.encode(' 12시 땡! 하루가 또 가네요.'))


[9349, 7888, 739, 7318, 376, 12557, 6824, 9108, 9028, 7098, 25856]


In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)


In [ ]:

steps = len(train_data) // batch_size + 1
print(steps)

370


In [ ]:
# pip install git+https://github.com/huggingface/transformers


In [ ]:
# pip install --upgrade git+https://github.com/huggingface/transformers.git

In [ ]:
#  pip install --upgrade git+https://github.com/huggingface/transformers.git

In [ ]:
# pip install transformers=="3.1.0" --no-dependencies

In [ ]:
EPOCHS = 3

for epoch in range(EPOCHS):
  epoch_loss = 0

  for batch in tqdm.notebook.tqdm(dataset, total=steps):
      with tf.GradientTape() as tape:
          result = model(batch, labels=batch)
          loss = result[0]
          batch_loss = tf.reduce_mean(loss)
          
      grads = tape.gradient(batch_loss, model.trainable_variables)
      adam.apply_gradients(zip(grads, model.trainable_variables))
      epoch_loss += batch_loss / steps

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

  0%|          | 0/370 [00:00<?, ?it/s]

InvalidArgumentError: ignored